In [2]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [1]:
from InitialPositions import *
from WaveFunctions import *
from Hamiltonians import *
from Optimizers import *
from sampler import Sampler
from metropolis import metro_util

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
# problem specific multi-use parameters
HO_shape = [1, 1, 1]
hard_core_r = 0.00433

config = {
    "name"        : "demo",
    "particles"   : 10,
    "dimensions"  : 3,
    
    "steps"       : {"sampled" : int(1.0E5), "equil" : int(1.0E3)},
    "metropolis"  : {"name" : "importance", "step_length" : 0.01},
    "optimizer"   : {"name" : "grid", "start" : 0.4, "end" : 0.6, "step" : 0.05},
    
    "initial"     : {"name" : "randomUniform", "radius" : hard_core_r},
    "wavefunc"    : {"name" : "simpleGaussian", "params" : 0.5, "HO shape" : HO_shape},
    "hamiltonian" : {"name" : "HO", "omega" : 1, "HO shape" : HO_shape},
    
    #"waveFunc"   : {"name" : "boson-corr", "alpha" : 0.5, "HO shape" : HO_shape, "hard-core-r" : hard_core_r}
}

In [5]:
initPos     = initPos_util.getInit(config)
metro_step  = metro_util(config)
hamiltonian = ham_util.getHam(config)

sampler   = Sampler(config)
waveFunc  = wf_util.getWf(config)
particles = initPos()

In [6]:
@tf.function
def test(pos):
    for i in range(100):
        pos = metro_step(pos, waveFunc)
        sampler.sample(pos, waveFunc, hamiltonian)
    return pos

In [7]:
pos = initPos()

In [9]:
%%time
pos = test(pos)

Wall time: 5.11 s


In [12]:
#50ms per 1000 steps -> 8.3 min per 10 million steps (100 particles) NO SAMPLING
50 * 10000000 / 1000 / 1000 / 60

8.333333333333334